# Set Up

## Import libraries and Webscrape data



In [ ]:
from bs4 import BeautifulSoup as bs
import requests
import pandas as pd
from matplotlib import pyplot as plt

In [ ]:
URLHEAD = "https://en.wikipedia.org/wiki/NFL_Top_100_Players_of_"
YEARS = []
for x in range(2010, 2020):
  YEARS.append(x)

In [ ]:
def get_data(year):
  #Get html based on what year
  r = requests.get(URLHEAD + str(year+1))
  soup = bs(r.content)
  #find the right table
  tables = soup.find_all('table')
  #from table headers get column names
  headers = tables[2].find('tbody').find_all('th')
  cols = []
  for header in headers:
    cols.append(header.text.strip())
  #Find rest of data
  tr = tables[2].find('tbody').find_all('tr')
  master = []
  row = []
  for ln in tr:
    #fill row with td    
    for td in ln.find_all('td'):
      row.append(td.text.strip())
    master.append(row)
    row = []
  master = master[1:]
  #make first row be column names
  master.insert(0,cols)
  return master

In [ ]:
#populate dictionary of all individual years and get consistent column names
df_list = {}
try:
  del temp_df
except:
  pass
for year in YEARS:
  temp_df = pd.DataFrame(get_data(year))
  cols = list(temp_df.iloc[0])
  temp_df = temp_df.iloc[1:]
  temp_df.columns=cols
  try:
    temp_df['Team'] = temp_df[str(year) + ' Team']
  except:
    temp_df['Team'] = temp_df[str(year) + ' team']
  temp_df.drop(temp_df.columns.difference(['Rank', 'Player', 'Position', 'Team', 'Year']), 1, inplace=True)
  temp_df['Year'] = year
  temp_df.head()
  df_list[year] = temp_df

In [ ]:
team_abv = {
    'New Orleans Saints':'NOR',
    'Atlanta Falcons':'ATL',
    'Carolina Panthers':'CAR',
    'Tampa Bay Buccaneers':'TB',
    'Indianapolis Colts':'IND',
    'Houston Texans':'HOU',
    'Tennessee Titans':'TEN',
    'Jacksonville Jaguars':'JAX',
    'Dallas Cowboys':'DAL',
    'New York Giants':'NYG',
    'Philadelphia Eagles':'PHI',
    'Washington Redskins':'WAS',
    'Washington Football Team':'WAS',
    'Baltimore Ravens':'BAL',
    'Cincinnati Bengals':'CIN',
    'Cleveland Browns':'CLV',
    'Pittsburgh Steelers':'PIT',
    'Buffalo Bills':'BUF',
    'Miami Dolphins':'MIA',
    'New England Patriots':'NE',
    'New York Jets':'NYJ',
    'Chicago Bears':'CHI',
    'Detroit Lions':'DET',
    'Green Bay Packers':'GB',
    'Minnesota Vikings':'MIN',
    'Denver Broncos':'DEN',
    'Kansas City Chiefs':'KC',
    'Las Vegas Raiders':'LV',
    'Oakland Raiders':'LV',
    'Los Angeles Chargers':'LAC',
    'San Diego Chargers':'LAC',
    'Arizona Cardinals':'ARI',
    'Los Angeles Rams':'LAR',
    'St. Louis Rams':'LAR',
    'San Francisco 49ers':'SF',
    'Seattle Seahawks':'SEA'
}

In [ ]:
position_abv = {
    'Quarterback':'QB',
    'Running back':'RB',
    'Wide Receiver':'WR',
    'Tight end':'TE',
    'Offensive Tackle':'OT',
    'Guard':'G',
    'Center':'C',
    'Defensive tackle':'DT',
    'Defensive end':'DE',
    'Cornerback':'CB',
    'Fullback':'FB',
    'Kicker':'K',
    'Free safety':'S',
    'Safety':'S',
    'linebacker':'LB',
    'Outside linebacker':'LB',
    'Strong Safety':'S'
}

In [ ]:
#concat all indv df into one master df
df = pd.DataFrame()
for idx in df_list:
  df = pd.concat([df,df_list[idx]])

for team in team_abv:
  df.loc[df['Team'].str.contains(team), 'Team'] = team_abv[team]

for pos in position_abv:
  df.loc[df['Position'].str.match(pos, case = False) , 'Position'] = position_abv[pos]


df['Rank'] = df['Rank'].astype('int32')
df['Points'] = 101 - df['Rank']
df.to_csv("Top 100 Players Master Dataset.csv")
df.head(20)

,Rank,Player,Position,Team,Year,Points
1,1,Tom Brady,QB,NE,2010,100
2,2,Peyton Manning,QB,IND,2010,99
3,3,Adrian Peterson,RB,MIN,2010,98
4,4,Ray Lewis,LB,BAL,2010,97
5,5,Ed Reed,S,BAL,2010,96
6,6,Troy Polamalu,S,PIT,2010,95
7,7,Andre Johnson,WR,HOU,2010,94
8,8,Darrelle Revis,CB,NYJ,2010,93
9,9,Drew Brees,QB,NOR,2010,92
10,10,Julius Peppers,DE,CHI,2010,91


## Up to here

In [ ]:
nola = df.loc[df['Team']=='NOR']
nola = nola.sort_values('Rank')
nola = nola.sort_values('Year')
nola.head(len(nola))

,Rank,Player,Position,Team,Year,Points
9,9,Drew Brees,QB,NOR,2010,92
55,55,Carl Nicks,G,NOR,2010,46
53,53,Marques Colston,WR,NOR,2010,48
37,37,Jonathan Vilma,LB,NOR,2010,64
34,34,Jahri Evans,G,NOR,2010,67
58,58,Jonathan Vilma,LB,NOR,2011,43
2,2,Drew Brees,QB,NOR,2011,99
86,86,Darren Sproles,RB,NOR,2011,15
76,76,Carl Nicks,G,NOR,2011,25
14,14,Jimmy Graham,TE,NOR,2011,87


In [ ]:
best_teams = df.groupby('Team').sum()
best_teams = best_teams.sort_values('Points', ascending = False)

best_teams.head(32)

,Rank,Year,Points
Team,,,
SEA,2358,100768,2692
KC,2574,102760,2577
PIT,1819,86628,2524
NE,1736,84589,2506
DAL,2603,100745,2447
NOR,1505,76566,2333
GB,2532,96686,2316
HOU,1171,66477,2162
BAL,2392,90607,2153


In [ ]:
most_points.head(20)

NameError: ignored

In [ ]:
most_points = df.groupby('Player').sum()
most_points = most_points.sort_values('Points', ascending= False)

most_points = most_points.iloc[0:21]

plt.bar(most_points.index, most_points['Points'])
plt.siz
plt.show()

In [ ]:
top10 = df.loc[df['Rank'] <= 10]
top10.tail(50)

In [ ]:
chi = top10.loc[top10['Team']=="NE"]
chi.head(12)

In [ ]:
team_top10 = top10.groupby('Team').count()
team_top10 = team_top10.sort_values('Points' , ascending= False)

team_top10.head(32)

In [ ]:
plt.bar(team_top10.index,team_top10['Rank'])
plt.show()

In [ ]:
master = []
pos_list = []
for pos in position_abv:
  if position_abv[pos] not in pos_list:
    pos_list.append(position_abv[pos])

for pos in pos_list:
  temp = []
  temp.append(pos)
  temp.append(len(df.loc[df['Position']==pos]))
  temp.append(min(df.loc[df['Position']==pos].Rank))
  temp.append(df.loc[df['Position']==pos].Rank.mean())
  master.append(temp)

pos_df = pd.DataFrame(master, columns = ['Position', 'Count', 'Highest Rank','Average Rank'])
pos_df = pos_df.set_index('Position')
pos_df = pos_df.sort_values('Count' , ascending = False)
pos_df.head(len(pos_df))

,Count,Highest Rank,Average Rank
Position,,,
WR,166,2,47.024096
QB,140,1,38.078571
LB,134,2,52.455224
RB,108,1,45.000000
DE,106,1,54.056604
CB,78,5,49.256410
DT,62,1,54.596774
S,62,5,59.903226
TE,54,7,53.833333


In [ ]:
master = []


team_list = []
for t in team_abv:
  if team_abv[t] not in team_list:
    team_list.append(team_abv[t])

for team in team_list:
  temp = []
  temp.append(team)
  temp.append(len(df.loc[df['Team']==team]))
  temp.append(min(df.loc[df['Team']==team].Rank))
  master.append(temp)

team_df = pd.DataFrame(master, columns = ['Team', 'Count','Highest Rank'])
team_df = team_df.set_index('Team')
team_df = team_df.sort_values("Count", ascending = False)
team_df.head(32)

,Count,Highest Rank
Team,,
KC,51,4
SEA,50,2
DAL,50,4
GB,48,1
BAL,45,1
PIT,43,2
NE,42,1
PHI,39,3
DEN,39,1
